In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

from __future__ import print_function
import pickle

In [3]:
url = 'https://www.the-numbers.com/movie/budgets/all/101'

user_agent = {'User-agent': 'Mozilla/5.0'}

response  = requests.get(url, headers = user_agent)

page = response.text

soup = BeautifulSoup(page,"lxml")

In [4]:
table = soup.find('table')
table

<table>
<tr><th> </th><th>Release<br/>Date</th><th>Movie</th><th>Production<br/>Budget</th><th>Domestic<br/>Gross</th><th>Worldwide<br/>Gross</th></tr>
<tr><td class="data">101</td>
<td><a href="/box-office-chart/daily/2015/02/06">Feb 6, 2015</a></td>
<td><b><a href="/movie/Jupiter-Ascending#tab=summary">Jupiter Ascending</a></b></td>
<td class="data"> $179,000,000</td>
<td class="data"> $47,482,519</td>
<td class="data"> $181,982,519</td>
</tr>
<tr><td class="data">102</td>
<td><a href="/box-office-chart/daily/2018/07/16">Jul 16, 2018</a></td>
<td><b><a href="/movie/Mission-Impossible-Fallout#tab=summary">Mission: ImpossibleâFallout</a></b></td>
<td class="data"> $178,000,000</td>
<td class="data"> $220,159,104</td>
<td class="data"> $787,176,729</td>
</tr>
<tr><td class="data">103</td>
<td><a href="/box-office-chart/daily/2016/05/18">May 18, 2016</a></td>
<td><b><a href="/movie/X-Men-Apocalypse-(2016)#tab=summary">X-Men: Apocalypse</a></b></td>
<td class="data"> $178,000,000</td>
<

In [5]:
rows = [row for row in table.find_all('tr')]

In [6]:
movies = {}

for row in rows[1:]:
    items = row.find_all('td')
    link = row.find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Feb 6, 2015': ['/box-office-chart/daily/2015/02/06',
  '101',
  'Feb 6, 2015',
  'Jupiter Ascending',
  '\xa0$179,000,000',
  '\xa0$47,482,519',
  '\xa0$181,982,519'],
 'Jul 16, 2018': ['/box-office-chart/daily/2018/07/16',
  '102',
  'Jul 16, 2018',
  'Mission: Impossibleâ\x80\x94Fallout',
  '\xa0$178,000,000',
  '\xa0$220,159,104',
  '\xa0$787,176,729'],
 'May 18, 2016': ['/box-office-chart/daily/2016/05/18',
  '103',
  'May 18, 2016',
  'X-Men: Apocalypse',
  '\xa0$178,000,000',
  '\xa0$155,442,489',
  '\xa0$542,537,546'],
 'Aug 9, 2018': ['/box-office-chart/daily/2018/08/09',
  '104',
  'Aug 9, 2018',
  'The Meg',
  '\xa0$178,000,000',
  '\xa0$145,443,742',
  '\xa0$527,370,715'],
 'May 28, 2014': ['/box-office-chart/daily/2014/05/28',
  '105',
  'May 28, 2014',
  'Edge of Tomorrow',
  '\xa0$178,000,000',
  '\xa0$100,206,256',
  '\xa0$367,028,980'],
 'Mar 6, 2019': ['/box-office-chart/daily/2019/03/06',
  '106',
  'Mar 6, 2019',
  'Captain Marvel',
  '\xa0$175,000,000',
  '\xa0$42

In [12]:
budget2 = pd.DataFrame(movies).T  #transpose
budget2.columns = ['link_stub','rank_all_movies','release','title', 
                    'budget', 'domestic_gross','worldwide_gross']

budget2.shape

(100, 7)

In [13]:
budget2['budget'] = budget2['budget'].str.replace('$', '')
budget2['budget'] = budget2['budget'].str.replace(',', '')
budget2['budget'] = budget2['budget'].astype(int)

In [14]:
budget2.head()

,link_stub,rank_all_movies,release,title,budget,domestic_gross,worldwide_gross
"Feb 6, 2015",/box-office-chart/daily/2015/02/06,101,"Feb 6, 2015",Jupiter Ascending,179000000,"$47,482,519","$181,982,519"
"Jul 16, 2018",/box-office-chart/daily/2018/07/16,102,"Jul 16, 2018",Mission: ImpossibleâFallout,178000000,"$220,159,104","$787,176,729"
"May 18, 2016",/box-office-chart/daily/2016/05/18,103,"May 18, 2016",X-Men: Apocalypse,178000000,"$155,442,489","$542,537,546"
"Aug 9, 2018",/box-office-chart/daily/2018/08/09,104,"Aug 9, 2018",The Meg,178000000,"$145,443,742","$527,370,715"
"May 28, 2014",/box-office-chart/daily/2014/05/28,105,"May 28, 2014",Edge of Tomorrow,178000000,"$100,206,256","$367,028,980"


In [9]:
budget2.to_csv('budget2.csv')